In [ ]:
import geopandas
import rasterio
import matplotlib.pyplot as plt
from shapely.geometry import Point
from rasterio.plot import show

import datetime as dt
import pandas as pd

import numpy as np
import glob, os

In [ ]:
print("foo")

# ATL03 Data elevation, sea surface freeboard

In [ ]:
import glob, os
import time

begin = time.time()

file = "lstm_model_31102024/LSTM_new_param_all_data_pred_cor_label_5km_new_sea_surf.csv"
print(file)
dup_removed = pd.read_csv(file, index_col=0)
day = 4
beam = "gt2r"
lon1 = -170.5 #-171.0
lon2 = -170.0
#track_field = #'track'#'track_x'
track_field = 'track_x'
dup_removed = dup_removed[(dup_removed['day'] == day) & (dup_removed[track_field] == beam)]

print("done")
h_diff_avg = dup_removed['height_diff_03_07'].mean()
print(h_diff_avg)
water_offset = 0.2+dup_removed['height_diff_03_07'].mean()
thinice_offset = 0.4+dup_removed['height_diff_03_07'].mean()
#dup_removed = dup_removed[(dup_removed.new_h_ref < new_h_ref_offset)]
print(dup_removed['new_h_ref'].mean())

from scipy.signal import savgol_filter
## Smooth data using Savitzky-Golay filter
#y_smoothed = savgol_filter(y, window_length=11, polyorder=2)

def smooth_line(data, w = 2):
    # Smooth the surface with the defined window size
    output = np.zeros(len(data))
    for n in range(0, len(data)):
        output[n] = np.nanmin(data[max(0, n-w):min(len(data), n+w+1)])
    return output
# Calculate the rolling average
dup_removed['new_h_ref_smooth'] = dup_removed['new_h_ref']
#dup_removed['new_h_ref_smooth'] = dup_removed['new_h_ref'].apply(lambda x: x if x <= water_offset else np.nan)
dup_removed['new_h_ref_smooth']  = smooth_line(dup_removed['new_h_ref_smooth'], w = 10000)
dup_removed['new_h_ref_smooth'] = dup_removed['new_h_ref_smooth'].interpolate(method='nearest', limit_direction='both', axis=0)
### freeboard calculation
dup_removed['freeboard_new_h_ref_smooth'] = dup_removed['h_cor_mean'] - dup_removed['new_h_ref_smooth']


#= dup_removed['new_h_ref'].rolling(window=window_size).min()
#= savgol_filter(dup_removed['new_h_ref'], window_length=10000, polyorder=2)
#display(data_copy)

# re-extract the data
thick_ice = dup_removed[(dup_removed.pred_label == 0)]#&(dup_removed.h_cor_mean >= thinice_offset)]
thin_ice = dup_removed[(dup_removed.pred_label == 1)]#&(dup_removed.h_cor_mean <= thinice_offset)]
water = dup_removed[(dup_removed.pred_label == 2)]#&(dup_removed.h_cor_mean <= water_offset)]

#26
#dup_removed = dup_removed[(dup_removed['lon'] >= -162.96) & (dup_removed['lon'] <= -162.7)]

#04
#dup_removed = dup_removed[(dup_removed['lon'] >= lon1) & (dup_removed['lon'] <= lon2)]

### ---------------- ###### ---------------- ###### ---------------- ###### ---------------- ###
dup_removed = dup_removed[(dup_removed['x_atc'] >= 28300000) & (dup_removed['x_atc'] <= 28550000)]

plt.figure(figsize = (12,4), dpi=300)
field = 'h_cor_mean'
plt.scatter(thick_ice.x_atc, thick_ice[field], s = 2, c='C0')
plt.scatter(thin_ice.x_atc, thin_ice[field], s = 2, c='C1')
plt.scatter(water.x_atc, water[field], s = 2, c='C2')
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('ATL03 along track lat (degree)')
plt.ylabel('elevation (m)')
plt.legend(['thick ice', 'thin ice', 'water'])
plt.show()

plt.figure(figsize = (12,4), dpi=300)
field = 'sea_surf_min_elev'
plt.scatter(dup_removed.x_atc, dup_removed[field]-h_diff_avg, s = 2, c='C2') # plot the elevation of thick ice points
field = 'sea_surf_avg_elev'
plt.scatter(dup_removed.x_atc, dup_removed[field]-h_diff_avg, s = 2, c='C1') # plot the elevation of thick ice points
field = 'sea_surf_min_dist'
plt.scatter(dup_removed.x_atc, dup_removed[field]-h_diff_avg, s = 2, c='C3') # plot the elevation of thick ice points
field2 = 'new_h_ref_smooth'#'new_h_ref'
plt.scatter(dup_removed.x_atc, dup_removed[field2]-h_diff_avg, s = 2, c='C0') # plot the elevation of thick ice points
#field3 = 'h_ref'
#plt.scatter(dup_removed.x_atc, dup_removed[field3], s = 2) # plot the elevation of thick ice points
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('Along-Track Distance (m)')
plt.ylabel('Sea Surface Elevation (m)')
plt.ylim(-0.4, 0.4)
plt.legend(['Minimum Elevation', 'Average Elevation', 'Nearest Minimum Elevation', 'NASA\'s Sea Surface Formula'])
#plt.savefig('ss04_1_v1.png')
#plt.savefig('ss26_1.png')
plt.savefig('ss04_1_v2.png')
plt.show()

plt.figure(figsize = (12,4), dpi=150)
field1 = 'new_h_ref_smooth'
plt.scatter(dup_removed.x_atc, dup_removed[field1]-h_diff_avg, s = 2) # plot the elevation of thick ice points
# field2 = 'new_h_ref'
# plt.scatter(dup_removed.x_atc, dup_removed[field2]-h_diff_avg, s = 2) # plot the elevation of thick ice points
field3 = 'h_ref'
plt.scatter(dup_removed.x_atc, dup_removed[field3]-h_diff_avg, s = 2) # plot the elevation of thick ice points
#plt.scatter(dup_removed.x_atc, dup_removed[field3]-dup_removed[field1], s = 2) # plot the elevation of thick ice points
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('Along-Track Distance (m)')
plt.ylim(-0.4, 0.4)
# plt.legend(['ATL03 Sea Surface smooth', 'ATL03 Sea Surface', 'ATL07  Sea Surface'])
plt.legend(['ATL03 Sea Surface', 'ATL07  Sea Surface'])
# plt.savefig('ss04_2_v1.png')
#plt.savefig('ss26_2.png')
plt.savefig('ss04_2_v2.png')
plt.show()
 
                    
del dup_removed

end = time.time() - begin
print("Total time needed ",end)
print("------------ Finish ------------")    


In [ ]:
import glob, os
import time

begin = time.time()

file = "lstm_model_31102024/LSTM_new_param_all_data_pred_cor_label_5km_new_sea_surf.csv"
print(file)
dup_removed = pd.read_csv(file, index_col=0)
day = 26
beam = "gt2r"
lon1 = -162.96 #-171.0
lon2 = -162.7
#track_field = #'track'#'track_x'
track_field = 'track_x'
dup_removed = dup_removed[(dup_removed['day'] == day) & (dup_removed[track_field] == beam)]

print("done")
h_diff_avg = dup_removed['height_diff_03_07'].mean()
print(h_diff_avg)
water_offset = 0.2+dup_removed['height_diff_03_07'].mean()
thinice_offset = 0.4+dup_removed['height_diff_03_07'].mean()
#dup_removed = dup_removed[(dup_removed.new_h_ref < new_h_ref_offset)]
print(dup_removed['new_h_ref'].mean())

from scipy.signal import savgol_filter
## Smooth data using Savitzky-Golay filter
#y_smoothed = savgol_filter(y, window_length=11, polyorder=2)

def smooth_line(data, w = 2):
    # Smooth the surface with the defined window size
    output = np.zeros(len(data))
    for n in range(0, len(data)):
        output[n] = np.nanmin(data[max(0, n-w):min(len(data), n+w+1)])
    return output
# Calculate the rolling average
dup_removed['new_h_ref_smooth'] = dup_removed['new_h_ref']
#dup_removed['new_h_ref_smooth'] = dup_removed['new_h_ref'].apply(lambda x: x if x <= water_offset else np.nan)
dup_removed['new_h_ref_smooth']  = smooth_line(dup_removed['new_h_ref_smooth'], w = 10000)
dup_removed['new_h_ref_smooth'] = dup_removed['new_h_ref_smooth'].interpolate(method='nearest', limit_direction='both', axis=0)
### freeboard calculation
dup_removed['freeboard_new_h_ref_smooth'] = dup_removed['h_cor_mean'] - dup_removed['new_h_ref_smooth']


#= dup_removed['new_h_ref'].rolling(window=window_size).min()
#= savgol_filter(dup_removed['new_h_ref'], window_length=10000, polyorder=2)
#display(data_copy)

# re-extract the data
thick_ice = dup_removed[(dup_removed.pred_label == 0)]#&(dup_removed.h_cor_mean >= thinice_offset)]
thin_ice = dup_removed[(dup_removed.pred_label == 1)]#&(dup_removed.h_cor_mean <= thinice_offset)]
water = dup_removed[(dup_removed.pred_label == 2)]#&(dup_removed.h_cor_mean <= water_offset)]

#26
#dup_removed = dup_removed[(dup_removed['lon'] >= -162.96) & (dup_removed['lon'] <= -162.7)]

#04
#dup_removed = dup_removed[(dup_removed['lon'] >= lon1) & (dup_removed['lon'] <= lon2)]

### ---------------- ###### ---------------- ###### ---------------- ###### ---------------- ###
dup_removed = dup_removed[(dup_removed['x_atc'] >= 28300000) & (dup_removed['x_atc'] <= 28550000)]

plt.figure(figsize = (12,4), dpi=300)
field = 'h_cor_mean'
plt.scatter(thick_ice.x_atc, thick_ice[field], s = 2, c='C0')
plt.scatter(thin_ice.x_atc, thin_ice[field], s = 2, c='C1')
plt.scatter(water.x_atc, water[field], s = 2, c='C2')
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('ATL03 along track lat (degree)')
plt.ylabel('elevation (m)')
plt.legend(['thick ice', 'thin ice', 'water'])
plt.show()

plt.figure(figsize = (12,4), dpi=300)
field = 'sea_surf_min_elev'
plt.scatter(dup_removed.x_atc, dup_removed[field]-h_diff_avg, s = 2, c='C2') # plot the elevation of thick ice points
field = 'sea_surf_avg_elev'
plt.scatter(dup_removed.x_atc, dup_removed[field]-h_diff_avg, s = 2, c='C1') # plot the elevation of thick ice points
field = 'sea_surf_min_dist'
plt.scatter(dup_removed.x_atc, dup_removed[field]-h_diff_avg, s = 2, c='C3') # plot the elevation of thick ice points
field2 = 'new_h_ref_smooth'#'new_h_ref'
plt.scatter(dup_removed.x_atc, dup_removed[field2]-h_diff_avg, s = 2, c='C0') # plot the elevation of thick ice points
#field3 = 'h_ref'
#plt.scatter(dup_removed.x_atc, dup_removed[field3], s = 2) # plot the elevation of thick ice points
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('Along-Track Distance (m)')
plt.ylabel('Sea Surface Elevation (m)')
plt.ylim(-0.4, 0.4)
plt.legend(['Minimum Elevation', 'Average Elevation', 'Nearest Minimum Elevation', 'NASA\'s Sea Surface Formula'])
# plt.savefig('ss26_1_v1.png')
#plt.savefig('ss26_1.png')
plt.savefig('ss26_1_v2.png')
plt.show()

plt.figure(figsize = (12,4), dpi=150)
field1 = 'new_h_ref_smooth'
plt.scatter(dup_removed.x_atc, dup_removed[field1]-h_diff_avg, s = 2) # plot the elevation of thick ice points
# field2 = 'new_h_ref'
# plt.scatter(dup_removed.x_atc, dup_removed[field2]-h_diff_avg, s = 2) # plot the elevation of thick ice points
field3 = 'h_ref'
plt.scatter(dup_removed.x_atc, dup_removed[field3]-h_diff_avg, s = 2) # plot the elevation of thick ice points
#plt.scatter(dup_removed.x_atc, dup_removed[field3]-dup_removed[field1], s = 2) # plot the elevation of thick ice points
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('Along-Track Distance (m)')
plt.ylim(-0.4, 0.4)
# plt.legend(['ATL03 Sea Surface smooth', 'ATL03 Sea Surface', 'ATL07  Sea Surface'])
plt.legend(['ATL03 Sea Surface', 'ATL07  Sea Surface'])
# plt.savefig('ss26_2_v1.png')
#plt.savefig('ss26_2.png')
plt.savefig('ss26_2_v2.png')
plt.show()
 
                    
del dup_removed

end = time.time() - begin
print("Total time needed ",end)
print("------------ Finish ------------")    


# ATL03 vs ATL03 ori vs ATL07 vs ATL10

In [ ]:
import glob, os
import time

file = "lstm_model_31102024/LSTM_new_param_all_data_pred_cor_label_5km_new_sea_surf.csv"
print(file)
dup_removed = pd.read_csv(file, index_col=0)
day = 4
beam = "gt2r"
lon1 = -170.5 #-171.0
lon2 = -170.0
track_field = 'track_x'
dup_removed = dup_removed[(dup_removed['day'] == day) & (dup_removed[track_field] == beam)]

dir07_path = "csv_Iqrah/"
dir03_path = "ATL03_Nov_2019_down2023/csv/"
begin = time.time()
beam03 = beam
print(beam03)
date = "201911" + str(day).zfill(2) 
print(date)

atl07_file = glob.glob(dir07_path + "ATL07-02_" + date + "*.csv")

print(atl07_file[0])
atl07_data = pd.read_csv(atl07_file[0], index_col=0)
atl07_data = atl07_data[atl07_data.beam == beam03]
atl07_data = atl07_data.sort_values(by=['x'])
atl07_data = atl07_data.reset_index(drop=True)

atl03_file = glob.glob(dir03_path + "ATL03_" + date + "*.csv")

print(atl03_file[0])
atl03_data = pd.read_csv(atl03_file[0], index_col=0)
atl03_data = atl03_data.sort_values(by=['x'])
atl03_data = atl03_data.reset_index(drop=True)
atl03_data = atl03_data[(atl03_data.h_cor_mean <= 15)]

h_diff_avg = dup_removed['height_diff_03_07'].mean()
print(h_diff_avg)
water_offset = 0.2+dup_removed['height_diff_03_07'].mean()
thinice_offset = 0.4+dup_removed['height_diff_03_07'].mean()
print(dup_removed['new_h_ref'].mean())

from scipy.signal import savgol_filter
window_size = 10000

def smooth_line(data, w = 2):
    # Smooth the surface with the defined window size
    output = np.zeros(len(data))
    for n in range(0, len(data)):
        output[n] = np.nanmin(data[max(0, n-w):min(len(data), n+w+1)])
    return output
# Calculate the rolling average
dup_removed['new_h_ref_smooth'] = dup_removed['new_h_ref'].apply(lambda x: x if x <= water_offset else np.nan)
dup_removed['new_h_ref_smooth']  = smooth_line(dup_removed['new_h_ref_smooth'], w = 10000)
dup_removed['new_h_ref_smooth'] = dup_removed['new_h_ref_smooth'].interpolate(method='nearest', limit_direction='both', axis=0)
### freeboard calculation
dup_removed['freeboard_new_h_ref_smooth'] = dup_removed['h_cor_mean'] - dup_removed['new_h_ref_smooth']

merged_data = dup_removed.copy()

atl07_data = atl07_data[(atl07_data['lon'] >= lon1) & (atl07_data['lon'] <= lon2)]
atl03_data = atl03_data[(atl03_data['lon'] >= lon1) & (atl03_data['lon'] <= lon2)]
merged_data = merged_data[(merged_data['lon'] >= lon1) & (merged_data['lon'] <= lon2)]

# re-extract the data
thick_ice = merged_data[(merged_data.pred_label == 0)]#&(merged_data.h_cor_mean >= thinice_offset)]
thin_ice = merged_data[(merged_data.pred_label == 1)]#&(merged_data.h_cor_mean >= water_offset)&(merged_data.h_cor_mean <= thinice_offset)]
water = merged_data[(merged_data.pred_label == 2)]#&(merged_data.h_cor_mean <= water_offset)]
# re-extract the data
thick_ice2 = atl07_data[(atl07_data.label == 2)]
thin_ice2 = atl07_data[(atl07_data.label == 1)]
water2 = atl07_data[(atl07_data.label == 0)]

# ice = merged_data[(merged_data.pred_label <= 1)]
# ice2 = atl07_data[(atl07_data.label >= 1)]
ice = merged_data[(merged_data.pred_label <= 2)]
ice2 = atl07_data[(atl07_data.label >= 2)]
#####--------------------------------------------------------------------------------------#####

print("Individual ATL03, ATL07, ATL10 Freeboard")

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.scatter(ice.lon, ice[field], s = 2, c='C0')
# plt.scatter(thick_ice.lon, thick_ice[field], s = 2, c='C0')
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.ylim(-0.5,3.5)
# plt.legend(['thick ice freeboard 2m atl03'], loc='upper right')
plt.legend(['freeboard 2m ATL03'], loc='upper right')
# plt.savefig('freeboard_03_1_v1') #4th
plt.savefig('freeboard_03_1_v2') #4th
# plt.savefig('freeboard_03') #26th
plt.show()


plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard'
# plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C0')
plt.scatter(ice2.lon, ice2[field], s = 2, c='C0')
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.ylim(-0.5,3.5)
# plt.legend(['thick ice freeboard atl07 nasa'],loc='upper right')
plt.legend(['freeboard Koo_NN(ATL07)'],loc='upper right')
# plt.savefig('freeboard_07_1_v1') #4th
plt.savefig('freeboard_07_1_v2') #4th
# plt.savefig('freeboard_07') #26th
plt.show()


plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_10'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C0')
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.legend(['thick ice freeboard atl10 nasa'],loc='upper right')
#plt.savefig('freeboard_10_1')
plt.show()

#####--------------------------------------------------------------------------------------#####

plt.figure(figsize = (12,4), dpi=300)
field = 'h_ref'
plt.scatter(atl07_data.lat, atl07_data[field], s = 2)
plt.scatter(merged_data.lat, merged_data[field]-merged_data['height_diff_03_07'].mean(), s = 2)
field = 'h_ref_10'
plt.scatter(atl07_data.lat, atl07_data[field], s = 2)
field = 'new_h_ref_smooth' 
plt.scatter(merged_data.lat, merged_data[field]-merged_data['height_diff_03_07'].mean(), s = 2)
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track long (degree)')
plt.ylabel('elevation (m)')
plt.legend(['SS atl07 nasa', 'merged ATL03 ATL07', 'SS atl10 nasa', 'new_h_ref_smooth'])
plt.show()

print("Comp sea surface")

plt.figure(figsize = (4,4), dpi=100)
field1 = 'h_ref'
field2 = 'new_h_ref_smooth' 
plt.scatter( merged_data[field2], merged_data[field1], s = 2)
plt.grid(linestyle = ':', linewidth = 0.5)
plt.ylabel('merged ATL03 ATL07 elevation (m)')
plt.xlabel('new_h_ref_smoothelevation (m)')
#plt.legend(['merged ATL03 ATL07', 'new_h_ref_smooth'])
plt.show()

plt.figure(figsize = (4,4), dpi=100)
field1 = 'h_ref'
field2 = 'h_ref_10' 
plt.scatter( atl07_data[field1], atl07_data[field2], s = 2)
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('atl07_data ATL10 elevation (m)')
plt.ylabel('atl07_data elevation (m)')
#plt.legend(['merged ATL03 ATL07', 'new_h_ref_smooth'])
plt.show()


print("ATL03 vs ATL07 Freeboard")

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.scatter(thick_ice.lon, thick_ice[field], s = 2, c='C0')
field = 'freeboard'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C1')
field = 'freeboard_10'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C2')
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.legend(['thick ice freeboard 2m atl03', 'thick ice freeboard atl07 nasa', 'thick ice freeboard atl10 nasa'])
plt.show()

#####--------------------------------------------------------------------------------------#####

print("Histogram ATL03, ATL07, ATL10 Freeboard")

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.hist(ice.lon, bins=40, alpha=0.5, label='Data 1', edgecolor='black')
#field = 'freeboard'
#plt.hist(thick_ice2.lon, bins=30, alpha=0.5, label='Data 2', edgecolor='red')
field = 'freeboard_10'
plt.hist(ice2.lon, bins=40, alpha=0.5, label='Data 3', color='green', edgecolor='black')
plt.xlabel('along track lat (degree)')
plt.ylabel('No. of data points')
plt.ylim(0,1500)
plt.legend(['freeboard 2m ATL03', 'freeboard Koo_NN(ATL07)'])
# plt.savefig('hist04_atl_03_10_v1')
#plt.savefig('hist26_atl_03_10')
plt.savefig('hist04_atl_03_10_v2')
plt.show()


# plt.figure(figsize = (12,4), dpi=300)
# field = 'freeboard_new_h_ref_smooth'
# plt.hist(ice.lon, bins=40, alpha=0.5, label='Data 1', edgecolor='black')
# #field = 'freeboard'
# #plt.hist(thick_ice2.lon, bins=30, alpha=0.5, label='Data 2', edgecolor='red')
# field = 'freeboard_10'
# plt.hist(ice2.lon, bins=40, alpha=0.5, label='Data 3', color='green', edgecolor='black')
# plt.xlabel('along track lat (degree)')
# plt.ylabel('No. of data points')
# plt.ylim(0, 1500)
# plt.legend(['ice freeboard 2m ATL03', 'ice freeboard ATL07/ATL10'])
# plt.show()

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.hist(ice.freeboard_new_h_ref_smooth, bins=50, histtype='step', ec='blue', linewidth=2)
field = 'freeboard'
plt.hist(ice2.freeboard, bins=50, alpha=1, histtype='step', ec='red', linewidth=2)
field = 'freeboard_10'
plt.hist(ice2.freeboard_10, bins=50, alpha=1, histtype='step', ec='green', linewidth=2)
plt.xlabel('Freeboard (m)')
plt.ylabel('No. of data points')
#plt.ylim(0, 5000)
#plt.ylim(0, 1200)
plt.xlim(0, 3.5)
plt.legend(['freeboard 2m ATL03', 'freeboard Koo_NN(ATL07)', 'freeboard ATL10'])
# plt.savefig('freeboard_hist04_atl_03_07_10_v1')
#plt.savefig('freeboard_hist26_atl_03_07_10')
plt.savefig('freeboard_hist04_atl_03_07_10_v2')
plt.show()

#merged_data.to_csv(file.replace(src_dir_path, dest_dir_path).replace(".csv", "_atl07_sea_surf_dr.csv"))
del dup_removed, merged_data


end = time.time() - begin
print("Total time needed ",end)
print("------------ Finish ------------")    


In [ ]:
import glob, os
import time

file = "lstm_model_31102024/LSTM_new_param_all_data_pred_cor_label_5km_new_sea_surf.csv"
print(file)
dup_removed = pd.read_csv(file, index_col=0)
day = 26
beam = "gt2r"
lon1 = -162.96 #-171.0
lon2 = -162.7
#track_field = #'track'#'track_x'
track_field = 'track_x'
dup_removed = dup_removed[(dup_removed['day'] == day) & (dup_removed[track_field] == beam)]

dir07_path = "csv_Iqrah/"
dir03_path = "ATL03_Nov_2019_down2023/csv/"
begin = time.time()

beam03 = beam
print(beam03)
date = "201911" + str(day).zfill(2) 
print(date)

#break

dup_removed = dup_removed.sort_values(by=['x_atc'])
dup_removed = dup_removed.reset_index(drop=True)


atl07_file = glob.glob(dir07_path + "ATL07-02_" + date + "*.csv")

print(atl07_file[0])
atl07_data = pd.read_csv(atl07_file[0], index_col=0)
atl07_data = atl07_data[atl07_data.beam == beam03]
atl07_data = atl07_data.sort_values(by=['x'])
atl07_data = atl07_data.reset_index(drop=True)

atl03_file = glob.glob(dir03_path + "ATL03_" + date + "*.csv")

print(atl03_file[0])
atl03_data = pd.read_csv(atl03_file[0], index_col=0)
atl03_data = atl03_data.sort_values(by=['x'])
atl03_data = atl03_data.reset_index(drop=True)
atl03_data = atl03_data[(atl03_data.h_cor_mean <= 15)]
#display(atl07_data)

#display(dup_removed)
h_diff_avg = dup_removed['height_diff_03_07'].mean()
print(h_diff_avg)
water_offset = 0.2+dup_removed['height_diff_03_07'].mean()
thinice_offset = 0.4+dup_removed['height_diff_03_07'].mean()
#dup_removed = dup_removed[(dup_removed.new_h_ref < new_h_ref_offset)]
print(dup_removed['new_h_ref'].mean())

from scipy.signal import savgol_filter
window_size = 10000
def smooth_line(data, w = 2):
    output = np.zeros(len(data))
    for n in range(0, len(data)):
        output[n] = np.nanmin(data[max(0, n-w):min(len(data), n+w+1)])
    return output
# Calculate the rolling average
dup_removed['new_h_ref_smooth'] = dup_removed['new_h_ref'].apply(lambda x: x if x <= water_offset else np.nan)
dup_removed['new_h_ref_smooth']  = smooth_line(dup_removed['new_h_ref_smooth'], w = 10000)
dup_removed['new_h_ref_smooth'] = dup_removed['new_h_ref_smooth'].interpolate(method='nearest', limit_direction='both', axis=0)
### freeboard calculation
dup_removed['freeboard_new_h_ref_smooth'] = dup_removed['h_cor_mean'] - dup_removed['new_h_ref_smooth']



merged_data = dup_removed.copy()

atl07_data = atl07_data[(atl07_data['lon'] >= lon1) & (atl07_data['lon'] <= lon2)]
atl03_data = atl03_data[(atl03_data['lon'] >= lon1) & (atl03_data['lon'] <= lon2)]
merged_data = merged_data[(merged_data['lon'] >= lon1) & (merged_data['lon'] <= lon2)]

# re-extract the data
thick_ice = merged_data[(merged_data.pred_label == 0)]#&(merged_data.h_cor_mean >= thinice_offset)]
thin_ice = merged_data[(merged_data.pred_label == 1)]#&(merged_data.h_cor_mean >= water_offset)&(merged_data.h_cor_mean <= thinice_offset)]
water = merged_data[(merged_data.pred_label == 2)]#&(merged_data.h_cor_mean <= water_offset)]
# re-extract the data
thick_ice2 = atl07_data[(atl07_data.label == 2)]
thin_ice2 = atl07_data[(atl07_data.label == 1)]
water2 = atl07_data[(atl07_data.label == 0)]

# ice = merged_data[(merged_data.pred_label <= 1)]
# ice2 = atl07_data[(atl07_data.label >= 1)]
ice = merged_data[(merged_data.pred_label <= 2)]
ice2 = atl07_data[(atl07_data.label >= 2)]
#####--------------------------------------------------------------------------------------#####

print("Individual ATL03, ATL07, ATL10 Freeboard")


plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.scatter(ice.lon, ice[field], s = 2, c='C0')
# plt.scatter(thick_ice.lon, thick_ice[field], s = 2, c='C0')
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.ylim(-0.5,3.5)
# plt.legend(['thick ice freeboard 2m atl03'], loc='upper right')
plt.legend(['freeboard 2m ATL03'], loc='upper right')
#plt.savefig('freeboard_03_1') #4th
# plt.savefig('freeboard_03_v1') #26th
plt.savefig('freeboard_03_v2') #26th
plt.show()


plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard'
# plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C0')
plt.scatter(ice2.lon, ice2[field], s = 2, c='C0')
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.ylim(-0.5,3.5)
# plt.legend(['thick ice freeboard atl07 nasa'],loc='upper right')
plt.legend(['freeboard Koo_NN(ATL07)'],loc='upper right')
#plt.savefig('freeboard_07_1') #4th
# plt.savefig('freeboard_07_v1') #26th
plt.savefig('freeboard_07_v2') #26th
plt.show()


plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_10'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C0')
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.legend(['thick ice freeboard atl10 nasa'],loc='upper right')
#plt.savefig('freeboard_10_1')
plt.show()

#####--------------------------------------------------------------------------------------#####

plt.figure(figsize = (12,4), dpi=300)
field = 'h_ref'
plt.scatter(atl07_data.lat, atl07_data[field], s = 2)
plt.scatter(merged_data.lat, merged_data[field]-merged_data['height_diff_03_07'].mean(), s = 2)
field = 'h_ref_10'
plt.scatter(atl07_data.lat, atl07_data[field], s = 2)
field = 'new_h_ref_smooth' 
plt.scatter(merged_data.lat, merged_data[field]-merged_data['height_diff_03_07'].mean(), s = 2)
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track long (degree)')
plt.ylabel('elevation (m)')
plt.legend(['SS atl07 nasa', 'merged ATL03 ATL07', 'SS atl10 nasa', 'new_h_ref_smooth'])
plt.show()

print("Comp sea surface")

plt.figure(figsize = (4,4), dpi=100)
field1 = 'h_ref'
field2 = 'new_h_ref_smooth' 
plt.scatter( merged_data[field2], merged_data[field1], s = 2)
plt.grid(linestyle = ':', linewidth = 0.5)
plt.ylabel('merged ATL03 ATL07 elevation (m)')
plt.xlabel('new_h_ref_smoothelevation (m)')
#plt.legend(['merged ATL03 ATL07', 'new_h_ref_smooth'])
plt.show()

plt.figure(figsize = (4,4), dpi=100)
field1 = 'h_ref'
field2 = 'h_ref_10' 
plt.scatter( atl07_data[field1], atl07_data[field2], s = 2)
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('atl07_data ATL10 elevation (m)')
plt.ylabel('atl07_data elevation (m)')
#plt.legend(['merged ATL03 ATL07', 'new_h_ref_smooth'])
plt.show()



print("ATL03 vs ATL07 Freeboard")

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.scatter(thick_ice.lon, thick_ice[field], s = 2, c='C0')
field = 'freeboard'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C1')
field = 'freeboard_10'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C2')
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.legend(['thick ice freeboard 2m atl03', 'thick ice freeboard atl07 nasa', 'thick ice freeboard atl10 nasa'])
plt.show()

#####--------------------------------------------------------------------------------------#####

print("Histogram ATL03, ATL07, ATL10 Freeboard")

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.hist(ice.lon, bins=40, alpha=0.5, label='Data 1', edgecolor='black')
#field = 'freeboard'
#plt.hist(thick_ice2.lon, bins=30, alpha=0.5, label='Data 2', edgecolor='red')
field = 'freeboard_10'
plt.hist(ice2.lon, bins=40, alpha=0.5, label='Data 3', color='green', edgecolor='black')
plt.xlabel('along track lat (degree)')
plt.ylabel('No. of data points')
plt.ylim(0,800)
plt.legend(['freeboard 2m ATL03', 'freeboard Koo_NN(ATL07)'])
#plt.savefig('hist04_atl_03_10')
# plt.savefig('hist26_atl_03_10_v1')
plt.savefig('hist26_atl_03_10_v2')
plt.show()


# plt.figure(figsize = (12,4), dpi=300)
# field = 'freeboard_new_h_ref_smooth'
# plt.hist(ice.lon, bins=40, alpha=0.5, label='Data 1', edgecolor='black')
# #field = 'freeboard'
# #plt.hist(thick_ice2.lon, bins=30, alpha=0.5, label='Data 2', edgecolor='red')
# field = 'freeboard_10'
# plt.hist(ice2.lon, bins=40, alpha=0.5, label='Data 3', color='green', edgecolor='black')
# plt.xlabel('along track lat (degree)')
# plt.ylabel('No. of data points')
# plt.ylim(0, 800)
# plt.legend(['ice freeboard 2m ATL03', 'ice freeboard ATL07/ATL10'])
# plt.show()

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.hist(ice.freeboard_new_h_ref_smooth, bins=50, histtype='step', ec='blue', linewidth=2)
field = 'freeboard'
plt.hist(ice2.freeboard, bins=50, alpha=1, histtype='step', ec='red', linewidth=2)
field = 'freeboard_10'
plt.hist(ice2.freeboard_10, bins=50, alpha=1, histtype='step', ec='green', linewidth=2)
plt.xlabel('Freeboard (m)')
plt.ylabel('No. of data points')
#plt.ylim(0, 5000)
#plt.ylim(0, 1200)
plt.xlim(0, 3.5)
plt.legend(['freeboard 2m ATL03', 'freeboard Koo_NN(ATL07)', 'freeboard ATL10'])
# plt.savefig('freeboard_hist04_atl_03_07_10')
# plt.savefig('freeboard_hist26_atl_03_07_10_v1')
plt.savefig('freeboard_hist26_atl_03_07_10_v2')
plt.show()

#merged_data.to_csv(file.replace(src_dir_path, dest_dir_path).replace(".csv", "_atl07_sea_surf_dr.csv"))
del dup_removed, merged_data


end = time.time() - begin
print("Total time needed ",end)
print("------------ Finish ------------")    


In [ ]:
import glob, os
import time

file = "LSTM_new_param_all_data_pred_cor_label_5km_new_sea_surf.csv"
print(file)
dup_removed = pd.read_csv(file, index_col=0)
day = 3
beam = "gt1r"
lon1 = -153.3
lon2 = -152.5
#track_field = #'track'#'track_x'
track_field = 'track_x'
dup_removed = dup_removed[(dup_removed['day'] == day) & (dup_removed[track_field] == beam)]

dir07_path = "csv_Iqrah/"
dir03_path = "ATL03_Nov_2019_down2023/csv/"
begin = time.time()

beam03 = beam
print(beam03)
date = "201911" + str(day).zfill(2) 
print(date)

#break

dup_removed = dup_removed.sort_values(by=['x_atc'])
dup_removed = dup_removed.reset_index(drop=True)


atl07_file = glob.glob(dir07_path + "ATL07-02_" + date + "*.csv")

print(atl07_file[0])
atl07_data = pd.read_csv(atl07_file[0], index_col=0)
atl07_data = atl07_data[atl07_data.beam == beam03]
atl07_data = atl07_data.sort_values(by=['x'])
atl07_data = atl07_data.reset_index(drop=True)

atl03_file = glob.glob(dir03_path + "ATL03_" + date + "*.csv")

print(atl03_file[0])
atl03_data = pd.read_csv(atl03_file[0], index_col=0)
atl03_data = atl03_data.sort_values(by=['x'])
atl03_data = atl03_data.reset_index(drop=True)
atl03_data = atl03_data[(atl03_data.h_cor_mean <= 15)]
#display(atl07_data)

#display(dup_removed)
h_diff_avg = dup_removed['height_diff_03_07'].mean()
print(h_diff_avg)
water_offset = 0.2+dup_removed['height_diff_03_07'].mean()
thinice_offset = 0.4+dup_removed['height_diff_03_07'].mean()
#dup_removed = dup_removed[(dup_removed.new_h_ref < new_h_ref_offset)]
print(dup_removed['new_h_ref'].mean())

from scipy.signal import savgol_filter
window_size = 10000
def smooth_line(data, w = 2):
    output = np.zeros(len(data))
    for n in range(0, len(data)):
        output[n] = np.nanmin(data[max(0, n-w):min(len(data), n+w+1)])
    return output
# Calculate the rolling average
dup_removed['new_h_ref_smooth'] = dup_removed['new_h_ref'].apply(lambda x: x if x <= water_offset else np.nan)
dup_removed['new_h_ref_smooth']  = smooth_line(dup_removed['new_h_ref_smooth'], w = 10000)
dup_removed['new_h_ref_smooth'] = dup_removed['new_h_ref_smooth'].interpolate(method='nearest', limit_direction='both', axis=0)
### freeboard calculation
dup_removed['freeboard_new_h_ref_smooth'] = dup_removed['h_cor_mean'] - dup_removed['new_h_ref_smooth']



merged_data = dup_removed.copy()

atl07_data = atl07_data[(atl07_data['lon'] >= lon1) & (atl07_data['lon'] <= lon2)]
atl03_data = atl03_data[(atl03_data['lon'] >= lon1) & (atl03_data['lon'] <= lon2)]
merged_data = merged_data[(merged_data['lon'] >= lon1) & (merged_data['lon'] <= lon2)]

# re-extract the data
thick_ice = merged_data[(merged_data.pred_label == 0)]#&(merged_data.h_cor_mean >= thinice_offset)]
thin_ice = merged_data[(merged_data.pred_label == 1)]#&(merged_data.h_cor_mean >= water_offset)&(merged_data.h_cor_mean <= thinice_offset)]
water = merged_data[(merged_data.pred_label == 2)]#&(merged_data.h_cor_mean <= water_offset)]
# re-extract the data
thick_ice2 = atl07_data[(atl07_data.label == 2)]
thin_ice2 = atl07_data[(atl07_data.label == 1)]
water2 = atl07_data[(atl07_data.label == 0)]

#####--------------------------------------------------------------------------------------#####

print("Individual ATL03, ATL07, ATL10 Freeboard")

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.scatter(thick_ice.lon, thick_ice[field], s = 2, c='C0')
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.ylim(-0.5,3.5)
plt.legend(['thick ice freeboard 2m atl03'], loc='upper right')
#plt.savefig('freeboard_03_1') #4th
# plt.savefig('freeboard_03') #26th
plt.show()


plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C0')
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.ylim(-0.5,3.5)
plt.legend(['thick ice freeboard atl07 nasa'],loc='upper right')
#plt.savefig('freeboard_07_1') #4th
# plt.savefig('freeboard_07') #26th
plt.show()


plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_10'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C0')
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.legend(['thick ice freeboard atl10 nasa'],loc='upper right')
#plt.savefig('freeboard_10_1')
plt.show()

#####--------------------------------------------------------------------------------------#####

plt.figure(figsize = (12,4), dpi=300)
field = 'h_ref'
plt.scatter(atl07_data.lat, atl07_data[field], s = 2)
plt.scatter(merged_data.lat, merged_data[field]-merged_data['height_diff_03_07'].mean(), s = 2)
field = 'h_ref_10'
plt.scatter(atl07_data.lat, atl07_data[field], s = 2)
field = 'new_h_ref_smooth' 
plt.scatter(merged_data.lat, merged_data[field]-merged_data['height_diff_03_07'].mean(), s = 2)
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track long (degree)')
plt.ylabel('elevation (m)')
plt.legend(['SS atl07 nasa', 'merged ATL03 ATL07', 'SS atl10 nasa', 'new_h_ref_smooth'])
plt.show()

print("Comp sea surface")

plt.figure(figsize = (4,4), dpi=100)
field1 = 'h_ref'
field2 = 'new_h_ref_smooth' 
plt.scatter( merged_data[field2], merged_data[field1], s = 2)
plt.grid(linestyle = ':', linewidth = 0.5)
plt.ylabel('merged ATL03 ATL07 elevation (m)')
plt.xlabel('new_h_ref_smoothelevation (m)')
#plt.legend(['merged ATL03 ATL07', 'new_h_ref_smooth'])
plt.show()

plt.figure(figsize = (4,4), dpi=100)
field1 = 'h_ref'
field2 = 'h_ref_10' 
plt.scatter( atl07_data[field1], atl07_data[field2], s = 2)
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('atl07_data ATL10 elevation (m)')
plt.ylabel('atl07_data elevation (m)')
#plt.legend(['merged ATL03 ATL07', 'new_h_ref_smooth'])
plt.show()



ice = merged_data[(merged_data.pred_label <= 1)]
ice2 = atl07_data[(atl07_data.label >= 1)]

print("ATL03 vs ATL07 Freeboard")

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.scatter(thick_ice.lon, thick_ice[field], s = 2, c='C0')
field = 'freeboard'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C1')
field = 'freeboard_10'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C2')
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.legend(['thick ice freeboard 2m atl03', 'thick ice freeboard atl07 nasa', 'thick ice freeboard atl10 nasa'])
plt.show()

#####--------------------------------------------------------------------------------------#####

print("Histogram ATL03, ATL07, ATL10 Freeboard")

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.hist(thick_ice.lon, bins=40, alpha=0.5, label='Data 1', edgecolor='black')
#field = 'freeboard'
#plt.hist(thick_ice2.lon, bins=30, alpha=0.5, label='Data 2', edgecolor='red')
field = 'freeboard_10'
plt.hist(thick_ice2.lon, bins=40, alpha=0.5, label='Data 3', color='green', edgecolor='black')
plt.xlabel('along track lat (degree)')
plt.ylabel('No. of data points')
plt.ylim(0,2000)
plt.legend(['thick ice freeboard 2m ATL03', 'thick ice freeboard ATL07/ATL10'])
#plt.savefig('hist04_atl_03_10')
#plt.savefig('hist26_atl_03_10')
plt.show()


plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.hist(ice.lon, bins=40, alpha=0.5, label='Data 1', edgecolor='black')
#field = 'freeboard'
#plt.hist(thick_ice2.lon, bins=30, alpha=0.5, label='Data 2', edgecolor='red')
field = 'freeboard_10'
plt.hist(ice2.lon, bins=40, alpha=0.5, label='Data 3', color='green', edgecolor='black')
plt.xlabel('along track lat (degree)')
plt.ylabel('No. of data points')
plt.ylim(0, 2000)
plt.legend(['ice freeboard 2m ATL03', 'ice freeboard ATL07/ATL10'])
plt.show()

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.hist(ice.freeboard_new_h_ref_smooth, bins=50, histtype='step', ec='blue', linewidth=2)
field = 'freeboard'
plt.hist(ice2.freeboard, bins=50, alpha=1, histtype='step', ec='red', linewidth=2)
field = 'freeboard_10'
plt.hist(ice2.freeboard_10, bins=50, alpha=1, histtype='step', ec='green', linewidth=2)
plt.xlabel('Freeboard (m)')
plt.ylabel('No. of data points')
#plt.ylim(0, 5000)
#plt.ylim(0, 1200)
plt.xlim(0, 3.5)
plt.legend(['ice freeboard 2m ATL03', 'ice freeboard Koo_NN(ATL07)', 'ice freeboard ATL10'])
plt.savefig('freeboard_hist04_atl_03_07_10')
#plt.savefig('freeboard_hist26_atl_03_07_10')
plt.show()

#merged_data.to_csv(file.replace(src_dir_path, dest_dir_path).replace(".csv", "_atl07_sea_surf_dr.csv"))
del dup_removed, merged_data


end = time.time() - begin
print("Total time needed ",end)
print("------------ Finish ------------")    


In [ ]:
import glob, os
import time

file = "LSTM_new_param_all_data_pred_cor_label_5km_new_sea_surf.csv"
print(file)
dup_removed = pd.read_csv(file, index_col=0)
day = 13
beam = "gt1r"
lon1 = -168.4
lon2 = -167.2
#track_field = #'track'#'track_x'
track_field = 'track_x'
dup_removed = dup_removed[(dup_removed['day'] == day) & (dup_removed[track_field] == beam)]

dir07_path = "csv_Iqrah/"
dir03_path = "ATL03_Nov_2019_down2023/csv/"
begin = time.time()

beam03 = beam
print(beam03)
date = "201911" + str(day).zfill(2) 
print(date)

#break

dup_removed = dup_removed.sort_values(by=['x_atc'])
dup_removed = dup_removed.reset_index(drop=True)


atl07_file = glob.glob(dir07_path + "ATL07-02_" + date + "*.csv")

print(atl07_file[0])
atl07_data = pd.read_csv(atl07_file[0], index_col=0)
atl07_data = atl07_data[atl07_data.beam == beam03]
atl07_data = atl07_data.sort_values(by=['x'])
atl07_data = atl07_data.reset_index(drop=True)

atl03_file = glob.glob(dir03_path + "ATL03_" + date + "*.csv")

print(atl03_file[0])
atl03_data = pd.read_csv(atl03_file[0], index_col=0)
atl03_data = atl03_data.sort_values(by=['x'])
atl03_data = atl03_data.reset_index(drop=True)
atl03_data = atl03_data[(atl03_data.h_cor_mean <= 15)]
#display(atl07_data)

#display(dup_removed)
h_diff_avg = dup_removed['height_diff_03_07'].mean()
print(h_diff_avg)
water_offset = 0.2+dup_removed['height_diff_03_07'].mean()
thinice_offset = 0.4+dup_removed['height_diff_03_07'].mean()
#dup_removed = dup_removed[(dup_removed.new_h_ref < new_h_ref_offset)]
print(dup_removed['new_h_ref'].mean())

from scipy.signal import savgol_filter
window_size = 10000
def smooth_line(data, w = 2):
    output = np.zeros(len(data))
    for n in range(0, len(data)):
        output[n] = np.nanmin(data[max(0, n-w):min(len(data), n+w+1)])
    return output
# Calculate the rolling average
dup_removed['new_h_ref_smooth'] = dup_removed['new_h_ref'].apply(lambda x: x if x <= water_offset else np.nan)
dup_removed['new_h_ref_smooth']  = smooth_line(dup_removed['new_h_ref_smooth'], w = 10000)
dup_removed['new_h_ref_smooth'] = dup_removed['new_h_ref_smooth'].interpolate(method='nearest', limit_direction='both', axis=0)
### freeboard calculation
dup_removed['freeboard_new_h_ref_smooth'] = dup_removed['h_cor_mean'] - dup_removed['new_h_ref_smooth']



merged_data = dup_removed.copy()

atl07_data = atl07_data[(atl07_data['lon'] >= lon1) & (atl07_data['lon'] <= lon2)]
atl03_data = atl03_data[(atl03_data['lon'] >= lon1) & (atl03_data['lon'] <= lon2)]
merged_data = merged_data[(merged_data['lon'] >= lon1) & (merged_data['lon'] <= lon2)]

# re-extract the data
thick_ice = merged_data[(merged_data.pred_label == 0)]#&(merged_data.h_cor_mean >= thinice_offset)]
thin_ice = merged_data[(merged_data.pred_label == 1)]#&(merged_data.h_cor_mean >= water_offset)&(merged_data.h_cor_mean <= thinice_offset)]
water = merged_data[(merged_data.pred_label == 2)]#&(merged_data.h_cor_mean <= water_offset)]
# re-extract the data
thick_ice2 = atl07_data[(atl07_data.label == 2)]
thin_ice2 = atl07_data[(atl07_data.label == 1)]
water2 = atl07_data[(atl07_data.label == 0)]

#####--------------------------------------------------------------------------------------#####

print("Individual ATL03, ATL07, ATL10 Freeboard")

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.scatter(thick_ice.lon, thick_ice[field], s = 2, c='C0')
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.ylim(-0.5,3.5)
plt.legend(['thick ice freeboard 2m atl03'], loc='upper right')
#plt.savefig('freeboard_03_1') #4th
# plt.savefig('freeboard_03') #26th
plt.show()


plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C0')
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.ylim(-0.5,3.5)
plt.legend(['thick ice freeboard atl07 nasa'],loc='upper right')
#plt.savefig('freeboard_07_1') #4th
# plt.savefig('freeboard_07') #26th
plt.show()


plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_10'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C0')
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.legend(['thick ice freeboard atl10 nasa'],loc='upper right')
#plt.savefig('freeboard_10_1')
plt.show()

#####--------------------------------------------------------------------------------------#####

plt.figure(figsize = (12,4), dpi=300)
field = 'h_ref'
plt.scatter(atl07_data.lat, atl07_data[field], s = 2)
plt.scatter(merged_data.lat, merged_data[field]-merged_data['height_diff_03_07'].mean(), s = 2)
field = 'h_ref_10'
plt.scatter(atl07_data.lat, atl07_data[field], s = 2)
field = 'new_h_ref_smooth' 
plt.scatter(merged_data.lat, merged_data[field]-merged_data['height_diff_03_07'].mean(), s = 2)
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track long (degree)')
plt.ylabel('elevation (m)')
plt.legend(['SS atl07 nasa', 'merged ATL03 ATL07', 'SS atl10 nasa', 'new_h_ref_smooth'])
plt.show()

print("Comp sea surface")

plt.figure(figsize = (4,4), dpi=100)
field1 = 'h_ref'
field2 = 'new_h_ref_smooth' 
plt.scatter( merged_data[field2], merged_data[field1], s = 2)
plt.grid(linestyle = ':', linewidth = 0.5)
plt.ylabel('merged ATL03 ATL07 elevation (m)')
plt.xlabel('new_h_ref_smoothelevation (m)')
#plt.legend(['merged ATL03 ATL07', 'new_h_ref_smooth'])
plt.show()

plt.figure(figsize = (4,4), dpi=100)
field1 = 'h_ref'
field2 = 'h_ref_10' 
plt.scatter( atl07_data[field1], atl07_data[field2], s = 2)
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('atl07_data ATL10 elevation (m)')
plt.ylabel('atl07_data elevation (m)')
#plt.legend(['merged ATL03 ATL07', 'new_h_ref_smooth'])
plt.show()



ice = merged_data[(merged_data.pred_label <= 1)]
ice2 = atl07_data[(atl07_data.label >= 1)]

print("ATL03 vs ATL07 Freeboard")

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.scatter(thick_ice.lon, thick_ice[field], s = 2, c='C0')
field = 'freeboard'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C1')
field = 'freeboard_10'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C2')
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.legend(['thick ice freeboard 2m atl03', 'thick ice freeboard atl07 nasa', 'thick ice freeboard atl10 nasa'])
plt.show()

#####--------------------------------------------------------------------------------------#####

print("Histogram ATL03, ATL07, ATL10 Freeboard")

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.hist(thick_ice.lon, bins=40, alpha=0.5, label='Data 1', edgecolor='black')
#field = 'freeboard'
#plt.hist(thick_ice2.lon, bins=30, alpha=0.5, label='Data 2', edgecolor='red')
field = 'freeboard_10'
plt.hist(thick_ice2.lon, bins=40, alpha=0.5, label='Data 3', color='green', edgecolor='black')
plt.xlabel('along track lat (degree)')
plt.ylabel('No. of data points')
plt.ylim(0,3000)
plt.legend(['thick ice freeboard 2m ATL03', 'thick ice freeboard ATL07/ATL10'])
#plt.savefig('hist04_atl_03_10')
#plt.savefig('hist26_atl_03_10')
plt.show()


plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.hist(ice.lon, bins=40, alpha=0.5, label='Data 1', edgecolor='black')
#field = 'freeboard'
#plt.hist(thick_ice2.lon, bins=30, alpha=0.5, label='Data 2', edgecolor='red')
field = 'freeboard_10'
plt.hist(ice2.lon, bins=40, alpha=0.5, label='Data 3', color='green', edgecolor='black')
plt.xlabel('along track lat (degree)')
plt.ylabel('No. of data points')
plt.ylim(0, 3000)
plt.legend(['ice freeboard 2m ATL03', 'ice freeboard ATL07/ATL10'])
plt.show()

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.hist(ice.freeboard_new_h_ref_smooth, bins=50, histtype='step', ec='blue', linewidth=2)
field = 'freeboard'
plt.hist(ice2.freeboard, bins=50, alpha=1, histtype='step', ec='red', linewidth=2)
field = 'freeboard_10'
plt.hist(ice2.freeboard_10, bins=50, alpha=1, histtype='step', ec='green', linewidth=2)
plt.xlabel('Freeboard (m)')
plt.ylabel('No. of data points')
#plt.ylim(0, 5000)
#plt.ylim(0, 1200)
plt.xlim(0, 3.5)
plt.legend(['ice freeboard 2m ATL03', 'ice freeboard Koo_NN(ATL07)', 'ice freeboard ATL10'])
plt.savefig('freeboard_hist04_atl_03_07_10')
#plt.savefig('freeboard_hist26_atl_03_07_10')
plt.show()

#merged_data.to_csv(file.replace(src_dir_path, dest_dir_path).replace(".csv", "_atl07_sea_surf_dr.csv"))
del dup_removed, merged_data


end = time.time() - begin
print("Total time needed ",end)
print("------------ Finish ------------")    


In [ ]:
import glob, os
import time

file = "LSTM_new_param_all_data_pred_cor_label_5km_new_sea_surf.csv"
print(file)
dup_removed = pd.read_csv(file, index_col=0)
day = 20
beam = "gt1r"
lon1 = -172.5
lon2 = -172.0
#track_field = #'track'#'track_x'
track_field = 'track_x'
dup_removed = dup_removed[(dup_removed['day'] == day) & (dup_removed[track_field] == beam)]

dir07_path = "csv_Iqrah/"
dir03_path = "ATL03_Nov_2019_down2023/csv/"
begin = time.time()

beam03 = beam
print(beam03)
date = "201911" + str(day).zfill(2) 
print(date)

#break

dup_removed = dup_removed.sort_values(by=['x_atc'])
dup_removed = dup_removed.reset_index(drop=True)


atl07_file = glob.glob(dir07_path + "ATL07-02_" + date + "*.csv")

print(atl07_file[0])
atl07_data = pd.read_csv(atl07_file[0], index_col=0)
atl07_data = atl07_data[atl07_data.beam == beam03]
atl07_data = atl07_data.sort_values(by=['x'])
atl07_data = atl07_data.reset_index(drop=True)

atl03_file = glob.glob(dir03_path + "ATL03_" + date + "*.csv")

print(atl03_file[0])
atl03_data = pd.read_csv(atl03_file[0], index_col=0)
atl03_data = atl03_data.sort_values(by=['x'])
atl03_data = atl03_data.reset_index(drop=True)
atl03_data = atl03_data[(atl03_data.h_cor_mean <= 15)]
#display(atl07_data)

#display(dup_removed)
h_diff_avg = dup_removed['height_diff_03_07'].mean()
print(h_diff_avg)
water_offset = 0.2+dup_removed['height_diff_03_07'].mean()
thinice_offset = 0.4+dup_removed['height_diff_03_07'].mean()
#dup_removed = dup_removed[(dup_removed.new_h_ref < new_h_ref_offset)]
print(dup_removed['new_h_ref'].mean())

from scipy.signal import savgol_filter
window_size = 10000
def smooth_line(data, w = 2):
    output = np.zeros(len(data))
    for n in range(0, len(data)):
        output[n] = np.nanmin(data[max(0, n-w):min(len(data), n+w+1)])
    return output
# Calculate the rolling average
dup_removed['new_h_ref_smooth'] = dup_removed['new_h_ref'].apply(lambda x: x if x <= water_offset else np.nan)
dup_removed['new_h_ref_smooth']  = smooth_line(dup_removed['new_h_ref_smooth'], w = 10000)
dup_removed['new_h_ref_smooth'] = dup_removed['new_h_ref_smooth'].interpolate(method='nearest', limit_direction='both', axis=0)
### freeboard calculation
dup_removed['freeboard_new_h_ref_smooth'] = dup_removed['h_cor_mean'] - dup_removed['new_h_ref_smooth']



merged_data = dup_removed.copy()

atl07_data = atl07_data[(atl07_data['lon'] >= lon1) & (atl07_data['lon'] <= lon2)]
atl03_data = atl03_data[(atl03_data['lon'] >= lon1) & (atl03_data['lon'] <= lon2)]
merged_data = merged_data[(merged_data['lon'] >= lon1) & (merged_data['lon'] <= lon2)]

# re-extract the data
thick_ice = merged_data[(merged_data.pred_label == 0)]#&(merged_data.h_cor_mean >= thinice_offset)]
thin_ice = merged_data[(merged_data.pred_label == 1)]#&(merged_data.h_cor_mean >= water_offset)&(merged_data.h_cor_mean <= thinice_offset)]
water = merged_data[(merged_data.pred_label == 2)]#&(merged_data.h_cor_mean <= water_offset)]
# re-extract the data
thick_ice2 = atl07_data[(atl07_data.label == 2)]
thin_ice2 = atl07_data[(atl07_data.label == 1)]
water2 = atl07_data[(atl07_data.label == 0)]

#####--------------------------------------------------------------------------------------#####

print("Individual ATL03, ATL07, ATL10 Freeboard")

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.scatter(thick_ice.lon, thick_ice[field], s = 2, c='C0')
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.ylim(-0.5,3.5)
plt.legend(['thick ice freeboard 2m atl03'], loc='upper right')
#plt.savefig('freeboard_03_1') #4th
# plt.savefig('freeboard_03') #26th
plt.show()


plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C0')
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.ylim(-0.5,3.5)
plt.legend(['thick ice freeboard atl07 nasa'],loc='upper right')
#plt.savefig('freeboard_07_1') #4th
# plt.savefig('freeboard_07') #26th
plt.show()


plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_10'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C0')
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.legend(['thick ice freeboard atl10 nasa'],loc='upper right')
#plt.savefig('freeboard_10_1')
plt.show()

#####--------------------------------------------------------------------------------------#####

plt.figure(figsize = (12,4), dpi=300)
field = 'h_ref'
plt.scatter(atl07_data.lat, atl07_data[field], s = 2)
plt.scatter(merged_data.lat, merged_data[field]-merged_data['height_diff_03_07'].mean(), s = 2)
field = 'h_ref_10'
plt.scatter(atl07_data.lat, atl07_data[field], s = 2)
field = 'new_h_ref_smooth' 
plt.scatter(merged_data.lat, merged_data[field]-merged_data['height_diff_03_07'].mean(), s = 2)
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track long (degree)')
plt.ylabel('elevation (m)')
plt.legend(['SS atl07 nasa', 'merged ATL03 ATL07', 'SS atl10 nasa', 'new_h_ref_smooth'])
plt.show()

print("Comp sea surface")

plt.figure(figsize = (4,4), dpi=100)
field1 = 'h_ref'
field2 = 'new_h_ref_smooth' 
plt.scatter( merged_data[field2], merged_data[field1], s = 2)
plt.grid(linestyle = ':', linewidth = 0.5)
plt.ylabel('merged ATL03 ATL07 elevation (m)')
plt.xlabel('new_h_ref_smoothelevation (m)')
#plt.legend(['merged ATL03 ATL07', 'new_h_ref_smooth'])
plt.show()

plt.figure(figsize = (4,4), dpi=100)
field1 = 'h_ref'
field2 = 'h_ref_10' 
plt.scatter( atl07_data[field1], atl07_data[field2], s = 2)
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('atl07_data ATL10 elevation (m)')
plt.ylabel('atl07_data elevation (m)')
#plt.legend(['merged ATL03 ATL07', 'new_h_ref_smooth'])
plt.show()



ice = merged_data[(merged_data.pred_label <= 1)]
ice2 = atl07_data[(atl07_data.label >= 1)]

print("ATL03 vs ATL07 Freeboard")

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.scatter(thick_ice.lon, thick_ice[field], s = 2, c='C0')
field = 'freeboard'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C1')
field = 'freeboard_10'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C2')
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.legend(['thick ice freeboard 2m atl03', 'thick ice freeboard atl07 nasa', 'thick ice freeboard atl10 nasa'])
plt.show()

#####--------------------------------------------------------------------------------------#####

print("Histogram ATL03, ATL07, ATL10 Freeboard")

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.hist(thick_ice.lon, bins=40, alpha=0.5, label='Data 1', edgecolor='black')
#field = 'freeboard'
#plt.hist(thick_ice2.lon, bins=30, alpha=0.5, label='Data 2', edgecolor='red')
field = 'freeboard_10'
plt.hist(thick_ice2.lon, bins=40, alpha=0.5, label='Data 3', color='green', edgecolor='black')
plt.xlabel('along track lat (degree)')
plt.ylabel('No. of data points')
plt.ylim(0,2000)
plt.legend(['thick ice freeboard 2m ATL03', 'thick ice freeboard ATL07/ATL10'])
#plt.savefig('hist04_atl_03_10')
#plt.savefig('hist26_atl_03_10')
plt.show()


plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.hist(ice.lon, bins=40, alpha=0.5, label='Data 1', edgecolor='black')
#field = 'freeboard'
#plt.hist(thick_ice2.lon, bins=30, alpha=0.5, label='Data 2', edgecolor='red')
field = 'freeboard_10'
plt.hist(ice2.lon, bins=40, alpha=0.5, label='Data 3', color='green', edgecolor='black')
plt.xlabel('along track lat (degree)')
plt.ylabel('No. of data points')
plt.ylim(0, 2000)
plt.legend(['ice freeboard 2m ATL03', 'ice freeboard ATL07/ATL10'])
plt.show()

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.hist(ice.freeboard_new_h_ref_smooth, bins=50, histtype='step', ec='blue', linewidth=2)
field = 'freeboard'
plt.hist(ice2.freeboard, bins=50, alpha=1, histtype='step', ec='red', linewidth=2)
field = 'freeboard_10'
plt.hist(ice2.freeboard_10, bins=50, alpha=1, histtype='step', ec='green', linewidth=2)
plt.xlabel('Freeboard (m)')
plt.ylabel('No. of data points')
#plt.ylim(0, 5000)
#plt.ylim(0, 1200)
plt.xlim(0, 3.5)
plt.legend(['ice freeboard 2m ATL03', 'ice freeboard Koo_NN(ATL07)', 'ice freeboard ATL10'])
plt.savefig('freeboard_hist04_atl_03_07_10')
#plt.savefig('freeboard_hist26_atl_03_07_10')
plt.show()

#merged_data.to_csv(file.replace(src_dir_path, dest_dir_path).replace(".csv", "_atl07_sea_surf_dr.csv"))
del dup_removed, merged_data


end = time.time() - begin
print("Total time needed ",end)
print("------------ Finish ------------")    


In [ ]:
import glob, os
import time

file = "LSTM_new_param_all_data_pred_cor_label_5km_new_sea_surf.csv"
print(file)
dup_removed = pd.read_csv(file, index_col=0)
day = 23
beam = "gt2r"
lon1 = -157.75
lon2 = -156.25
#track_field = #'track'#'track_x'
track_field = 'track_x'
dup_removed = dup_removed[(dup_removed['day'] == day) & (dup_removed[track_field] == beam)]

dir07_path = "csv_Iqrah/"
dir03_path = "ATL03_Nov_2019_down2023/csv/"
begin = time.time()

beam03 = beam
print(beam03)
date = "201911" + str(day).zfill(2) 
print(date)

#break

dup_removed = dup_removed.sort_values(by=['x_atc'])
dup_removed = dup_removed.reset_index(drop=True)


atl07_file = glob.glob(dir07_path + "ATL07-02_" + date + "*.csv")

print(atl07_file[0])
atl07_data = pd.read_csv(atl07_file[0], index_col=0)
atl07_data = atl07_data[atl07_data.beam == beam03]
atl07_data = atl07_data.sort_values(by=['x'])
atl07_data = atl07_data.reset_index(drop=True)

atl03_file = glob.glob(dir03_path + "ATL03_" + date + "*.csv")

print(atl03_file[0])
atl03_data = pd.read_csv(atl03_file[0], index_col=0)
atl03_data = atl03_data.sort_values(by=['x'])
atl03_data = atl03_data.reset_index(drop=True)
atl03_data = atl03_data[(atl03_data.h_cor_mean <= 15)]
#display(atl07_data)

#display(dup_removed)
h_diff_avg = dup_removed['height_diff_03_07'].mean()
print(h_diff_avg)
water_offset = 0.2+dup_removed['height_diff_03_07'].mean()
thinice_offset = 0.4+dup_removed['height_diff_03_07'].mean()
#dup_removed = dup_removed[(dup_removed.new_h_ref < new_h_ref_offset)]
print(dup_removed['new_h_ref'].mean())

from scipy.signal import savgol_filter
window_size = 10000
def smooth_line(data, w = 2):
    output = np.zeros(len(data))
    for n in range(0, len(data)):
        output[n] = np.nanmin(data[max(0, n-w):min(len(data), n+w+1)])
    return output
# Calculate the rolling average
dup_removed['new_h_ref_smooth'] = dup_removed['new_h_ref'].apply(lambda x: x if x <= water_offset else np.nan)
dup_removed['new_h_ref_smooth']  = smooth_line(dup_removed['new_h_ref_smooth'], w = 10000)
dup_removed['new_h_ref_smooth'] = dup_removed['new_h_ref_smooth'].interpolate(method='nearest', limit_direction='both', axis=0)
### freeboard calculation
dup_removed['freeboard_new_h_ref_smooth'] = dup_removed['h_cor_mean'] - dup_removed['new_h_ref_smooth']



merged_data = dup_removed.copy()

atl07_data = atl07_data[(atl07_data['lon'] >= lon1) & (atl07_data['lon'] <= lon2)]
atl03_data = atl03_data[(atl03_data['lon'] >= lon1) & (atl03_data['lon'] <= lon2)]
merged_data = merged_data[(merged_data['lon'] >= lon1) & (merged_data['lon'] <= lon2)]

# re-extract the data
thick_ice = merged_data[(merged_data.pred_label == 0)]#&(merged_data.h_cor_mean >= thinice_offset)]
thin_ice = merged_data[(merged_data.pred_label == 1)]#&(merged_data.h_cor_mean >= water_offset)&(merged_data.h_cor_mean <= thinice_offset)]
water = merged_data[(merged_data.pred_label == 2)]#&(merged_data.h_cor_mean <= water_offset)]
# re-extract the data
thick_ice2 = atl07_data[(atl07_data.label == 2)]
thin_ice2 = atl07_data[(atl07_data.label == 1)]
water2 = atl07_data[(atl07_data.label == 0)]

#####--------------------------------------------------------------------------------------#####

print("Individual ATL03, ATL07, ATL10 Freeboard")

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.scatter(thick_ice.lon, thick_ice[field], s = 2, c='C0')
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.ylim(-0.5,3.5)
plt.legend(['thick ice freeboard 2m atl03'], loc='upper right')
#plt.savefig('freeboard_03_1') #4th
# plt.savefig('freeboard_03') #26th
plt.show()


plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C0')
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.ylim(-0.5,3.5)
plt.legend(['thick ice freeboard atl07 nasa'],loc='upper right')
#plt.savefig('freeboard_07_1') #4th
# plt.savefig('freeboard_07') #26th
plt.show()


plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_10'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C0')
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.legend(['thick ice freeboard atl10 nasa'],loc='upper right')
#plt.savefig('freeboard_10_1')
plt.show()

#####--------------------------------------------------------------------------------------#####

plt.figure(figsize = (12,4), dpi=300)
field = 'h_ref'
plt.scatter(atl07_data.lat, atl07_data[field], s = 2)
plt.scatter(merged_data.lat, merged_data[field]-merged_data['height_diff_03_07'].mean(), s = 2)
field = 'h_ref_10'
plt.scatter(atl07_data.lat, atl07_data[field], s = 2)
field = 'new_h_ref_smooth' 
plt.scatter(merged_data.lat, merged_data[field]-merged_data['height_diff_03_07'].mean(), s = 2)
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track long (degree)')
plt.ylabel('elevation (m)')
plt.legend(['SS atl07 nasa', 'merged ATL03 ATL07', 'SS atl10 nasa', 'new_h_ref_smooth'])
plt.show()

print("Comp sea surface")

plt.figure(figsize = (4,4), dpi=100)
field1 = 'h_ref'
field2 = 'new_h_ref_smooth' 
plt.scatter( merged_data[field2], merged_data[field1], s = 2)
plt.grid(linestyle = ':', linewidth = 0.5)
plt.ylabel('merged ATL03 ATL07 elevation (m)')
plt.xlabel('new_h_ref_smoothelevation (m)')
#plt.legend(['merged ATL03 ATL07', 'new_h_ref_smooth'])
plt.show()

plt.figure(figsize = (4,4), dpi=100)
field1 = 'h_ref'
field2 = 'h_ref_10' 
plt.scatter( atl07_data[field1], atl07_data[field2], s = 2)
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('atl07_data ATL10 elevation (m)')
plt.ylabel('atl07_data elevation (m)')
#plt.legend(['merged ATL03 ATL07', 'new_h_ref_smooth'])
plt.show()



ice = merged_data[(merged_data.pred_label <= 1)]
ice2 = atl07_data[(atl07_data.label >= 1)]

print("ATL03 vs ATL07 Freeboard")

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.scatter(thick_ice.lon, thick_ice[field], s = 2, c='C0')
field = 'freeboard'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C1')
field = 'freeboard_10'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C2')
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.legend(['thick ice freeboard 2m atl03', 'thick ice freeboard atl07 nasa', 'thick ice freeboard atl10 nasa'])
plt.show()

#####--------------------------------------------------------------------------------------#####

print("Histogram ATL03, ATL07, ATL10 Freeboard")

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.hist(thick_ice.lon, bins=40, alpha=0.5, label='Data 1', edgecolor='black')
#field = 'freeboard'
#plt.hist(thick_ice2.lon, bins=30, alpha=0.5, label='Data 2', edgecolor='red')
field = 'freeboard_10'
plt.hist(thick_ice2.lon, bins=40, alpha=0.5, label='Data 3', color='green', edgecolor='black')
plt.xlabel('along track lat (degree)')
plt.ylabel('No. of data points')
plt.ylim(0,4000)
plt.legend(['thick ice freeboard 2m ATL03', 'thick ice freeboard ATL07/ATL10'])
#plt.savefig('hist04_atl_03_10')
#plt.savefig('hist26_atl_03_10')
plt.show()


plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.hist(ice.lon, bins=40, alpha=0.5, label='Data 1', edgecolor='black')
#field = 'freeboard'
#plt.hist(thick_ice2.lon, bins=30, alpha=0.5, label='Data 2', edgecolor='red')
field = 'freeboard_10'
plt.hist(ice2.lon, bins=40, alpha=0.5, label='Data 3', color='green', edgecolor='black')
plt.xlabel('along track lat (degree)')
plt.ylabel('No. of data points')
plt.ylim(0, 4000)
plt.legend(['ice freeboard 2m ATL03', 'ice freeboard ATL07/ATL10'])
plt.show()

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.hist(ice.freeboard_new_h_ref_smooth, bins=50, histtype='step', ec='blue', linewidth=2)
field = 'freeboard'
plt.hist(ice2.freeboard, bins=50, alpha=1, histtype='step', ec='red', linewidth=2)
field = 'freeboard_10'
plt.hist(ice2.freeboard_10, bins=50, alpha=1, histtype='step', ec='green', linewidth=2)
plt.xlabel('Freeboard (m)')
plt.ylabel('No. of data points')
#plt.ylim(0, 5000)
#plt.ylim(0, 1200)
plt.xlim(0, 3.5)
plt.legend(['ice freeboard 2m ATL03', 'ice freeboard Koo_NN(ATL07)', 'ice freeboard ATL10'])
plt.savefig('freeboard_hist04_atl_03_07_10')
#plt.savefig('freeboard_hist26_atl_03_07_10')
plt.show()

#merged_data.to_csv(file.replace(src_dir_path, dest_dir_path).replace(".csv", "_atl07_sea_surf_dr.csv"))
del dup_removed, merged_data


end = time.time() - begin
print("Total time needed ",end)
print("------------ Finish ------------")    


In [ ]:
#

import glob, os
import time

file = "LSTM_new_param_all_data_pred_cor_label_5km_new_sea_surf.csv"
print(file)
dup_removed = pd.read_csv(file, index_col=0)
day = 4
beam = "gt2r"
lon1 = -170.5 #-171.0
lon2 = -170.0
#track_field = #'track'#'track_x'
track_field = 'track_x'
dup_removed = dup_removed[(dup_removed['day'] == day) & (dup_removed[track_field] == beam)]

dir07_path = "csv_Iqrah/"
#filelist1 = sorted(glob.glob(dir_path1 + "ATL03_" + "*_dr.csv"))
#filelist07 = sorted(glob.glob(dir07_path + "ATL07-" + "*.csv"))

dir03_path = "ATL03_Nov_2019_down2023/csv/"

#display(filelist)
#display(filelist07)

begin = time.time()

#ATL03_20191103184432_05780510_gt1r_labeled_10m_z_score_5km_atl07_sea_surf_dr_new_sea_surf.csv

beam03 = beam
print(beam03)
date = "201911" + str(day).zfill(2) 
print(date)

#break

dup_removed = dup_removed.sort_values(by=['x_atc'])
dup_removed = dup_removed.reset_index(drop=True)


atl07_file = glob.glob(dir07_path + "ATL07-02_" + date + "*.csv")

print(atl07_file[0])
atl07_data = pd.read_csv(atl07_file[0], index_col=0)
atl07_data = atl07_data[atl07_data.beam == beam03]
atl07_data = atl07_data.sort_values(by=['x'])
atl07_data = atl07_data.reset_index(drop=True)
#display(atl07_data)

#ATL03_20191113191053_07310510_gt1r.csv
atl03_file = glob.glob(dir03_path + "ATL03_" + date + "*.csv")

print(atl03_file[0])
atl03_data = pd.read_csv(atl03_file[0], index_col=0)
atl03_data = atl03_data.sort_values(by=['x'])
atl03_data = atl03_data.reset_index(drop=True)
atl03_data = atl03_data[(atl03_data.h_cor_mean <= 15)]
#display(atl07_data)

#display(dup_removed)
h_diff_avg = dup_removed['height_diff_03_07'].mean()
print(h_diff_avg)
water_offset = 0.2+dup_removed['height_diff_03_07'].mean()
thinice_offset = 0.4+dup_removed['height_diff_03_07'].mean()
#dup_removed = dup_removed[(dup_removed.new_h_ref < new_h_ref_offset)]
print(dup_removed['new_h_ref'].mean())

from scipy.signal import savgol_filter
## Smooth data using Savitzky-Golay filter
#y_smoothed = savgol_filter(y, window_length=11, polyorder=2)

# Define the window size
window_size = 10000

def smooth_line(data, w = 2):
    # Smooth the surface with the defined window size
    output = np.zeros(len(data))
    for n in range(0, len(data)):
        output[n] = np.nanmin(data[max(0, n-w):min(len(data), n+w+1)])
    return output
# Calculate the rolling average
dup_removed['new_h_ref_smooth'] = dup_removed['new_h_ref'].apply(lambda x: x if x <= water_offset else np.nan)
dup_removed['new_h_ref_smooth']  = smooth_line(dup_removed['new_h_ref_smooth'], w = 10000)
dup_removed['new_h_ref_smooth'] = dup_removed['new_h_ref_smooth'].interpolate(method='nearest', limit_direction='both', axis=0)
### freeboard calculation
dup_removed['freeboard_new_h_ref_smooth'] = dup_removed['h_cor_mean'] - dup_removed['new_h_ref_smooth']



merged_data = dup_removed.copy()

#         atl07_data = atl07_data[(atl07_data['lon'] >= -162.96) & (atl07_data['lon'] <= -162.7)]
#         atl03_data = atl03_data[(atl03_data['lon'] >= -162.96) & (atl03_data['lon'] <= -162.7)]
#         merged_data = merged_data[(merged_data['lon'] >= -162.96) & (merged_data['lon'] <= -162.7)]

atl07_data = atl07_data[(atl07_data['lon'] >= lon1) & (atl07_data['lon'] <= lon2)]
atl03_data = atl03_data[(atl03_data['lon'] >= lon1) & (atl03_data['lon'] <= lon2)]
merged_data = merged_data[(merged_data['lon'] >= lon1) & (merged_data['lon'] <= lon2)]

# re-extract the data
thick_ice = merged_data[(merged_data.pred_label == 0)]#&(merged_data.h_cor_mean >= thinice_offset)]
thin_ice = merged_data[(merged_data.pred_label == 1)]#&(merged_data.h_cor_mean >= water_offset)&(merged_data.h_cor_mean <= thinice_offset)]
water = merged_data[(merged_data.pred_label == 2)&(merged_data.h_cor_mean <= water_offset)]
# re-extract the data
thick_ice2 = atl07_data[(atl07_data.label == 2)]
thin_ice2 = atl07_data[(atl07_data.label == 1)]
water2 = atl07_data[(atl07_data.label == 0)]

ice = merged_data[(merged_data.pred_label <= 1)]
ice2 = atl07_data[(atl07_data.label >= 1)]

print("ATL03 vs ATL07 Freeboard")

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.scatter(thick_ice.lon, thick_ice[field], s = 2, c='C0')
field = 'freeboard'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C1')
field = 'freeboard_10'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C2')
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.legend(['thick ice freeboard 2m atl03', 'thick ice freeboard atl07 nasa', 'thick ice freeboard atl10 nasa'])
plt.show()

#####--------------------------------------------------------------------------------------#####

print("Histogram ATL03, ATL07, ATL10 Freeboard")

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.hist(thick_ice.lon, bins=40, alpha=0.5, label='Data 1', edgecolor='black')
#field = 'freeboard'
#plt.hist(thick_ice2.lon, bins=30, alpha=0.5, label='Data 2', edgecolor='red')
field = 'freeboard_10'
plt.hist(thick_ice2.lon, bins=40, alpha=0.5, label='Data 3', color='green', edgecolor='black')
plt.xlabel('along track lat (degree)')
plt.ylabel('No. of data points')
plt.ylim(0,1500)
plt.legend(['thick ice freeboard 2m ATL03', 'thick ice freeboard ATL07/ATL10'])
#plt.savefig('hist04_atl_03_10')
#plt.savefig('hist26_atl_03_10')
plt.show()


plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.hist(ice.lon, bins=40, alpha=0.5, label='Data 1', edgecolor='black')
#field = 'freeboard'
#plt.hist(thick_ice2.lon, bins=30, alpha=0.5, label='Data 2', edgecolor='red')
field = 'freeboard_10'
plt.hist(ice2.lon, bins=40, alpha=0.5, label='Data 3', color='green', edgecolor='black')
plt.xlabel('along track lat (degree)')
plt.ylabel('No. of data points')
plt.ylim(0, 1500)
plt.legend(['ice freeboard 2m ATL03', 'ice freeboard ATL07/ATL10'])
plt.show()

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.hist(ice.freeboard_new_h_ref_smooth, bins=50, histtype='step', ec='blue', linewidth=2)
field = 'freeboard'
plt.hist(ice2.freeboard, bins=50, alpha=1, histtype='step', ec='red', linewidth=2)
field = 'freeboard_10'
plt.hist(ice2.freeboard_10, bins=50, alpha=1, histtype='step', ec='green', linewidth=2)
plt.xlabel('Freeboard (m)')
plt.ylabel('No. of data points')
#plt.ylim(0, 5000)
#plt.ylim(0, 1200)
plt.xlim(0, 3.5)
plt.legend(['ice freeboard 2m ATL03', 'ice freeboard Koo_NN(ATL07)', 'ice freeboard ATL10'])
plt.savefig('freeboard_hist04_atl_03_07_10')
#plt.savefig('freeboard_hist26_atl_03_07_10')
plt.show()

#         plt.figure(figsize = (12,4), dpi=300)
#         field = 'freeboard_new_h_ref_smooth'
#         plt.hist(thick_ice.freeboard_new_h_ref_smooth, bins=40, alpha=0.5, label='Data 1', edgecolor='black')
#         plt.xlabel('Freeboard (m)')
#         plt.ylabel('No. of data points')
#         #plt.ylim(0, 5000)
#         plt.ylim(0, 2000)
#         plt.legend(['thick ice freeboard 2m ATL03'])
#         #plt.savefig('freeboard_hist04_atl_03')
#         plt.savefig('freeboard_hist26_atl_03')
#         plt.show()

#         plt.figure(figsize = (12,4), dpi=300)
#         field = 'freeboard'
#         plt.hist(thick_ice2.freeboard, bins=40, alpha=0.5, label='Data 2', color='red', edgecolor='black')
#         plt.xlabel('Freeboard (m)')
#         plt.ylabel('No. of data points')
#         #plt.ylim(0, 5000)
#         plt.ylim(0, 2000)
#         plt.legend(['thick ice freeboard ATL07'])
#         #plt.savefig('freeboard_hist04_atl_07')
#         plt.savefig('freeboard_hist26_atl_07')
#         plt.show()

#         plt.figure(figsize = (12,4), dpi=300)
#         field = 'freeboard_10'
#         plt.hist(thick_ice2.freeboard_10, bins=40, alpha=0.5, label='Data 3', color='green', edgecolor='black')
#         plt.xlabel('Freeboard (m)')
#         plt.ylabel('No. of data points')
#         #plt.ylim(0, 5000)
#         plt.ylim(0, 2000)
#         plt.legend(['thick ice freeboard ATL10'])
#         #plt.savefig('freeboard_hist04_atl_10')
#         plt.savefig('freeboard_hist26_atl_10')
#         plt.show()

#bin 0.05, 0.1, 0.15

#####--------------------------------------------------------------------------------------#####

#         print("Individual ATL03, ATL07, ATL10 Freeboard")

#         plt.figure(figsize = (12,4), dpi=300)
#         field = 'freeboard_new_h_ref_smooth'
#         plt.scatter(thick_ice.lon, thick_ice[field], s = 2, c='C0')
#         plt.xlabel('along track lat (degree)')
#         plt.ylabel('elevation (m)')
#         plt.legend(['thick ice freeboard 2m atl03'], loc='upper right')
#         plt.savefig('freeboard_03_1')
#         plt.show()


#         plt.figure(figsize = (12,4), dpi=300)
#         field = 'freeboard'
#         plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C0')
#         plt.grid(linestyle = ':', linewidth = 0.5)
#         plt.xlabel('along track lat (degree)')
#         plt.ylabel('elevation (m)')
#         plt.legend(['thick ice freeboard atl07 nasa'],loc='upper right')
#         plt.savefig('freeboard_07_1')
#         plt.show()


#         plt.figure(figsize = (12,4), dpi=300)
#         field = 'freeboard_10'
#         plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C0')
#         plt.grid(linestyle = ':', linewidth = 0.5)
#         plt.xlabel('along track lat (degree)')
#         plt.ylabel('elevation (m)')
#         plt.legend(['thick ice freeboard atl10 nasa'],loc='upper right')
#         plt.savefig('freeboard_10_1')
#         plt.show()


#merged_data.to_csv(file.replace(src_dir_path, dest_dir_path).replace(".csv", "_atl07_sea_surf_dr.csv"))
del dup_removed, merged_data

end = time.time() - start
print("Time per file is ",end)
print("------------ File done ------------")

#break

end = time.time() - begin
print("Total time needed ",end)
print("------------ Finish ------------")    


In [ ]:
#

import glob, os
import time

file = "LSTM_new_param_all_data_pred_cor_label_5km_new_sea_surf.csv"
print(file)
dup_removed = pd.read_csv(file, index_col=0)
day = 26
beam = "gt2r"
lon1 = -162.96 #-171.0
lon2 = -162.7
#track_field = #'track'#'track_x'
track_field = 'track_x'
dup_removed = dup_removed[(dup_removed['day'] == day) & (dup_removed[track_field] == beam)]

dir07_path = "csv_Iqrah/"
#filelist1 = sorted(glob.glob(dir_path1 + "ATL03_" + "*_dr.csv"))
#filelist07 = sorted(glob.glob(dir07_path + "ATL07-" + "*.csv"))

dir03_path = "ATL03_Nov_2019_down2023/csv/"

#display(filelist)
#display(filelist07)

begin = time.time()

#ATL03_20191103184432_05780510_gt1r_labeled_10m_z_score_5km_atl07_sea_surf_dr_new_sea_surf.csv

beam03 = beam
print(beam03)
date = "201911" + str(day).zfill(2) 
print(date)

#break


dup_removed = dup_removed.sort_values(by=['x_atc'])
dup_removed = dup_removed.reset_index(drop=True)


atl07_file = glob.glob(dir07_path + "ATL07-02_" + date + "*.csv")
print(atl07_file[0])
atl07_data = pd.read_csv(atl07_file[0], index_col=0)
atl07_data = atl07_data[atl07_data.beam == beam03]
atl07_data = atl07_data.sort_values(by=['x'])
atl07_data = atl07_data.reset_index(drop=True)
#display(atl07_data)

#ATL03_20191113191053_07310510_gt1r.csv
atl03_file = glob.glob(dir03_path + "ATL03_" + date + "*.csv")
print(atl03_file[0])
atl03_data = pd.read_csv(atl03_file[0], index_col=0)
atl03_data = atl03_data.sort_values(by=['x'])
atl03_data = atl03_data.reset_index(drop=True)
atl03_data = atl03_data[(atl03_data.h_cor_mean <= 15)]
#display(atl07_data)

#display(dup_removed)
h_diff_avg = dup_removed['height_diff_03_07'].mean()
print(h_diff_avg)
water_offset = 0.2+dup_removed['height_diff_03_07'].mean()
thinice_offset = 0.4+dup_removed['height_diff_03_07'].mean()
#dup_removed = dup_removed[(dup_removed.new_h_ref < new_h_ref_offset)]
print(dup_removed['new_h_ref'].mean())

from scipy.signal import savgol_filter
## Smooth data using Savitzky-Golay filter
#y_smoothed = savgol_filter(y, window_length=11, polyorder=2)

# Define the window size
window_size = 10000

def smooth_line(data, w = 2):
    # Smooth the surface with the defined window size
    output = np.zeros(len(data))
    for n in range(0, len(data)):
        output[n] = np.nanmin(data[max(0, n-w):min(len(data), n+w+1)])
    return output
# Calculate the rolling average
dup_removed['new_h_ref_smooth'] = dup_removed['new_h_ref'].apply(lambda x: x if x <= water_offset else np.nan)
dup_removed['new_h_ref_smooth']  = smooth_line(dup_removed['new_h_ref_smooth'], w = 10000)
dup_removed['new_h_ref_smooth'] = dup_removed['new_h_ref_smooth'].interpolate(method='nearest', limit_direction='both', axis=0)
### freeboard calculation
dup_removed['freeboard_new_h_ref_smooth'] = dup_removed['h_cor_mean'] - dup_removed['new_h_ref_smooth']



merged_data = dup_removed.copy()

atl07_data = atl07_data[(atl07_data['lon'] >= lon1) & (atl07_data['lon'] <= lon2)]
atl03_data = atl03_data[(atl03_data['lon'] >= lon1) & (atl03_data['lon'] <= lon2)]
merged_data = merged_data[(merged_data['lon'] >= lon1) & (merged_data['lon'] <= lon2)]

#         atl07_data = atl07_data[(atl07_data['lon'] >= -171.0) & (atl07_data['lon'] <= -170.5)]
#         atl03_data = atl03_data[(atl03_data['lon'] >= -171.0) & (atl03_data['lon'] <= -170.5)]
#         merged_data = merged_data[(merged_data['lon'] >= -171.0) & (merged_data['lon'] <= -170.5)]

# re-extract the data
thick_ice = merged_data[(merged_data.pred_label == 0)]#&(merged_data.h_cor_mean >= thinice_offset)]
thin_ice = merged_data[(merged_data.pred_label == 1)]#&(merged_data.h_cor_mean >= water_offset)&(merged_data.h_cor_mean <= thinice_offset)]
water = merged_data[(merged_data.pred_label == 2)&(merged_data.h_cor_mean <= water_offset)]
# re-extract the data
thick_ice2 = atl07_data[(atl07_data.label == 2)]
thin_ice2 = atl07_data[(atl07_data.label == 1)]
water2 = atl07_data[(atl07_data.label == 0)]

ice = merged_data[(merged_data.pred_label <= 1)]
ice2 = atl07_data[(atl07_data.label >= 1)]

print("ATL03 vs ATL07 Freeboard")

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.scatter(thick_ice.lon, thick_ice[field], s = 2, c='C0')
field = 'freeboard'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C1')
field = 'freeboard_10'
plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C2')
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track lat (degree)')
plt.ylabel('elevation (m)')
plt.legend(['thick ice freeboard 2m atl03', 'thick ice freeboard atl07 nasa', 'thick ice freeboard atl10 nasa'])
plt.show()

#####--------------------------------------------------------------------------------------#####

print("Histogram ATL03, ATL07, ATL10 Freeboard")

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.hist(thick_ice.lon, bins=40, alpha=0.5, label='Data 1', edgecolor='black')
#field = 'freeboard'
#plt.hist(thick_ice2.lon, bins=30, alpha=0.5, label='Data 2', edgecolor='red')
field = 'freeboard_10'
plt.hist(thick_ice2.lon, bins=40, alpha=0.5, label='Data 3', color='green', edgecolor='black')
plt.xlabel('along track lat (degree)')
plt.ylabel('No. of data points')
plt.ylim(0, 800)
plt.legend(['thick ice freeboard 2m ATL03', 'thick ice freeboard ATL07/ATL10'])
#plt.savefig('hist04_atl_03_10')
#plt.savefig('hist26_atl_03_10')
plt.show()


plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.hist(ice.lon, bins=40, alpha=0.5, label='Data 1', edgecolor='black')
#field = 'freeboard'
#plt.hist(thick_ice2.lon, bins=30, alpha=0.5, label='Data 2', edgecolor='red')
field = 'freeboard_10'
plt.hist(ice2.lon, bins=40, alpha=0.5, label='Data 3', color='green', edgecolor='black')
plt.xlabel('along track lat (degree)')
plt.ylabel('No. of data points')
plt.ylim(0, 800)
plt.legend(['ice freeboard 2m ATL03', 'ice freeboard ATL07/ATL10'])
plt.show()

plt.figure(figsize = (12,4), dpi=300)
field = 'freeboard_new_h_ref_smooth'
plt.hist(ice.freeboard_new_h_ref_smooth, bins=50, histtype='step', ec='blue', linewidth=2)
field = 'freeboard'
plt.hist(ice2.freeboard, bins=50, alpha=1, histtype='step', ec='red', linewidth=2)
field = 'freeboard_10'
plt.hist(ice2.freeboard_10, bins=50, alpha=1, histtype='step', ec='green', linewidth=2)
plt.xlabel('Freeboard (m)')
plt.ylabel('No. of data points')
#plt.ylim(0, 5000)
#plt.ylim(0, 1200)
plt.xlim(0, 3.5)
plt.legend(['ice freeboard 2m ATL03', 'ice freeboard Koo_NN(ATL07)', 'ice freeboard ATL10'])
#plt.savefig('freeboard_hist04_atl_03_07_10')
#         plt.savefig('freeboard_hist26_atl_03_07_10')
plt.show()

#         plt.figure(figsize = (12,4), dpi=300)
#         field = 'freeboard_new_h_ref_smooth'
#         plt.hist(thick_ice.freeboard_new_h_ref_smooth, bins=40, alpha=0.5, label='Data 1', edgecolor='black')
#         plt.xlabel('Freeboard (m)')
#         plt.ylabel('No. of data points')
#         #plt.ylim(0, 5000)
#         plt.ylim(0, 2000)
#         plt.legend(['thick ice freeboard 2m ATL03'])
#         #plt.savefig('freeboard_hist04_atl_03')
#         plt.savefig('freeboard_hist26_atl_03')
#         plt.show()

#         plt.figure(figsize = (12,4), dpi=300)
#         field = 'freeboard'
#         plt.hist(thick_ice2.freeboard, bins=40, alpha=0.5, label='Data 2', color='red', edgecolor='black')
#         plt.xlabel('Freeboard (m)')
#         plt.ylabel('No. of data points')
#         #plt.ylim(0, 5000)
#         plt.ylim(0, 2000)
#         plt.legend(['thick ice freeboard ATL07'])
#         #plt.savefig('freeboard_hist04_atl_07')
#         plt.savefig('freeboard_hist26_atl_07')
#         plt.show()

#         plt.figure(figsize = (12,4), dpi=300)
#         field = 'freeboard_10'
#         plt.hist(thick_ice2.freeboard_10, bins=40, alpha=0.5, label='Data 3', color='green', edgecolor='black')
#         plt.xlabel('Freeboard (m)')
#         plt.ylabel('No. of data points')
#         #plt.ylim(0, 5000)
#         plt.ylim(0, 2000)
#         plt.legend(['thick ice freeboard ATL10'])
#         #plt.savefig('freeboard_hist04_atl_10')
#         plt.savefig('freeboard_hist26_atl_10')
#         plt.show()

#bin 0.05, 0.1, 0.15

#####--------------------------------------------------------------------------------------#####

#         print("Individual ATL03, ATL07, ATL10 Freeboard")

#         plt.figure(figsize = (12,4), dpi=300)
#         field = 'freeboard_new_h_ref_smooth'
#         plt.scatter(thick_ice.lon, thick_ice[field], s = 2, c='C0')
#         plt.xlabel('along track lat (degree)')
#         plt.ylabel('elevation (m)')
#         plt.legend(['thick ice freeboard 2m atl03'], loc='upper right')
#         plt.savefig('freeboard_03_1')
#         plt.show()


#         plt.figure(figsize = (12,4), dpi=300)
#         field = 'freeboard'
#         plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C0')
#         plt.grid(linestyle = ':', linewidth = 0.5)
#         plt.xlabel('along track lat (degree)')
#         plt.ylabel('elevation (m)')
#         plt.legend(['thick ice freeboard atl07 nasa'],loc='upper right')
#         plt.savefig('freeboard_07_1')
#         plt.show()


#         plt.figure(figsize = (12,4), dpi=300)
#         field = 'freeboard_10'
#         plt.scatter(thick_ice2.lon, thick_ice2[field], s = 2, c='C0')
#         plt.grid(linestyle = ':', linewidth = 0.5)
#         plt.xlabel('along track lat (degree)')
#         plt.ylabel('elevation (m)')
#         plt.legend(['thick ice freeboard atl10 nasa'],loc='upper right')
#         plt.savefig('freeboard_10_1')
#         plt.show()


#merged_data.to_csv(file.replace(src_dir_path, dest_dir_path).replace(".csv", "_atl07_sea_surf_dr.csv"))
del dup_removed, merged_data

end = time.time() - begin
print("Total time needed ",end)
print("------------ Finish ------------")    


## S2 with Duplicate removed IS2

In [ ]:
############################################ part 2 ############################################

from rembg import remove
import cv2
import time

#path = 's2_tiff/'
file = 's2_tiff/s2_vis_39_20191120T200529_20191120T200529_T01CFV.tif' #glob.glob(path + "*_label.tif")[1]
#filelist = sorted(glob.glob(path + "s2_vis_" + "*.tif"))
#for file in filelist:
#print(file)
t0 = time.time()
img = rasterio.open(file, crs = 'EPSG:3976')

raw = img.read(1)
raw[raw < 0] = 0

ori = img.read()
ori[ori < 0] = 0
ori = ori.swapaxes(0,1)
ori = ori.swapaxes(1,2)

img_array = shadow_cloud_removal(ori)
img_array[img_array < 0] = 0
img_array = remove(img_array)
img_array = rgba2rgb(img_array, background=(0,0,0))  

#df = pd.read_csv('ATL03_labeled_data/labeled_sea_freeboard_10m_5km/ATL03_20191103184432_05780510_T05CMR_gt1r_labeled_10m_5km.csv', index_col=0)
#df = pd.read_csv('ATL03_labeled_data/labeled_sea_freeboard_10m_5km/ATL03_20191120191952_08380510_T01CFV_gt1r_labeled_10m_5km.csv', index_col=0)

#df = pd.read_csv('ATL03_labeled_data/labeled_10m_5km_height_cor_min_sea/duplicate_removed/ATL03_20191120191952_08380510_gt1r_labeled_10m_z_score_5km_min_sea_dr.csv', index_col=0)
df = pd.read_csv('ATL03_labeled_data/labeled_10m_5km_height_cor_avg_sea/duplicate_removed/ATL03_20191120191952_08380510_gt1r_labeled_10m_z_score_5km_avg_sea_dr.csv', index_col=0)

#df = pd.read_csv('ATL03_labeled_data/labeled_10m_5km_corrected/ATL03_20191120191952_08380510_gt1r_labeled_10m_z_score_5km.csv', index_col=0)

print("max sea surface", df.sea_surf_min_elev.max())
thick_ice = df[(df.label == 0)]# & (df.h_cor_mean > df.sea_surf_min_elev.max())]
thin_ice = df[(df.label == 1)]# & (df.h_cor_mean <= df.sea_surf_min_elev.max())]
open_water = df[(df.label == 2)]# & (df.h_cor_mean <= df.sea_surf_min_elev.max())]
data_copy = thick_ice.append(thin_ice.append(open_water))
plt.figure(figsize=(10,10))
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(ori, cv2.COLOR_BGR2RGB))
plt.scatter(df["pix_x"], df["pix_y"], c = df["label"], s=1, cmap = "jet")
plt.title("Origilan image")
plt.subplot(1, 2, 2)
plt.imshow(img_array, cmap ="gray")
plt.scatter(df["pix_x"], df["pix_y"], c = df["label"], s=1, cmap = "jet")
plt.title("Segmented image")
plt.show()

t1 = time.time()
total = t1-t0
print("time is ", total)
########################################### part 3 ############################################
plt.figure(figsize=(10,10))
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(ori, cv2.COLOR_BGR2RGB))
plt.scatter(df["pix_x"], df["pix_y"], c = df["h_cor_mean"], s=1, cmap = "jet")
plt.title("elev -> h_cor_mean image")
plt.subplot(1, 2, 2)
plt.imshow(img_array, cmap ="gray")
plt.scatter(df["pix_x"], df["pix_y"], c = df["h_cor_mean"], s=1, cmap = "jet")
plt.title("elev -> h_cor_mean image")
plt.show()

############################################ part 4 ############################################
plt.figure(figsize = (12,4), dpi=150)
#field = 'elev'
plt.scatter(data_copy.x_atc, data_copy['h_cor_mean'], s = 2) # plot the elevation of thick ice points
plt.scatter(data_copy.x_atc, data_copy['sea_surf_min_elev'], s = 2)
plt.scatter(data_copy.x_atc, data_copy['freeboard_min_elev'], s = 2)
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track distance (m)')
plt.ylabel('elevation (m)')
plt.legend(['h_cor_mean -> elev', 'sea_surf_min_elev', 'freeboard_min_elev'])
plt.show()

############################################ part 5 ############################################

# re-extract the data
thick_ice = (data_copy.label == 0) # thick_ice masking
thin_ice = (data_copy.label == 1) # thin_ice masking
water = (data_copy.label== 2) # open water masking

plt.figure(figsize = (12,4), dpi=150)
field = 'h_cor_mean'
plt.scatter(data_copy.x_atc[thick_ice], data_copy[field][thick_ice], s = 2) # plot the elevation of thick ice points
plt.scatter(data_copy.x_atc[thin_ice], data_copy[field][thin_ice], s = 2) # plot the elevation of thin ice points
plt.scatter(data_copy.x_atc[water], data_copy[field][water], s = 2) # plot the elevation of water points
field2 = 'sea_surf_min_elev'
plt.scatter(data_copy.x_atc[thick_ice], data_copy[field2][thick_ice], s = 2) # plot the elevation of thick ice points
plt.scatter(data_copy.x_atc[thin_ice], data_copy[field2][thin_ice], s = 2) # plot the elevation of thin ice points
plt.scatter(data_copy.x_atc[water], data_copy[field2][water], s = 2) # plot the elevation of water points
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track distance (m)')
plt.ylabel('elevation (m)')
plt.legend(['thick ice', 'thin ice', 'water', 'thick ice sea_surf_min_elev', 'thin ice sea_surf_min_elev', 'water sea_surf_min_elev'])
plt.show()

plt.figure(figsize = (12,4), dpi=150)
field = 'freeboard_min_elev'
plt.scatter(data_copy.x_atc[thick_ice], data_copy[field][thick_ice], s = 2) # plot the elevation of thick ice points
plt.scatter(data_copy.x_atc[thin_ice], data_copy[field][thin_ice], s = 2) # plot the elevation of thin ice points
plt.scatter(data_copy.x_atc[water], data_copy[field][water], s = 2) # plot the elevation of water points
field2 = 'sea_surf_min_elev'
plt.scatter(data_copy.x_atc[thick_ice], data_copy[field2][thick_ice], s = 2) # plot the elevation of thick ice points
plt.scatter(data_copy.x_atc[thin_ice], data_copy[field2][thin_ice], s = 2) # plot the elevation of thin ice points
plt.scatter(data_copy.x_atc[water], data_copy[field2][water], s = 2) # plot the elevation of water points
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track distance (m)')
plt.ylabel('freeboard_min_elev (m)')
plt.legend(['thick ice freeboard', 'thin ice freeboard', 'water freeboard', 'thick ice sea_surf_min_elev', 'thin ice sea_surf_min_elev', 'water sea_surf_min_elev'])
plt.show()

plt.figure(figsize = (12,4), dpi=150)
field = 'h_cor_mean'
plt.scatter(data_copy.x_atc[thick_ice], data_copy[field][thick_ice], s = 2) # plot the elevation of thick ice points
plt.scatter(data_copy.x_atc[thin_ice], data_copy[field][thin_ice], s = 2) # plot the elevation of thin ice points
plt.scatter(data_copy.x_atc[water], data_copy[field][water], s = 2) # plot the elevation of water points
field2 = 'freeboard_min_elev'
plt.scatter(data_copy.x_atc[thick_ice], data_copy[field2][thick_ice], s = 2) # plot the elevation of thick ice points
plt.scatter(data_copy.x_atc[thin_ice], data_copy[field2][thin_ice], s = 2) # plot the elevation of thin ice points
plt.scatter(data_copy.x_atc[water], data_copy[field2][water], s = 2) # plot the elevation of water points
plt.grid(linestyle = ':', linewidth = 0.5)
plt.xlabel('along track distance (m)')
plt.ylabel('elev (m)')
plt.legend(['thick ice', 'thin ice', 'water', 'thick ice freeboard', 'thin ice freeboard', 'water freeboard'])
plt.show()


data_copy
